# Welcome to Building Kevin Tutorial

This notebook aims to guide you how to build an AI programmer using LLMs.
We will look into how to prompt LLMs with proper context, send instructions to update existing codes and check for lint errors.


In [1]:
!pip install openai langchain langchain-core langchain-community langchain-openai

### Step 1: Connect to LLM API via Langchain
The first step is to make sure that we are able to invoke LLM API using Langchain. Langchain supports different LLMS include OpenAI and Llama via Ollama. We will show them both for purpose of demonstration.

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [12]:
# Test connection to OpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

current_model = ChatOpenAI(model="gpt-4o-mini",
                          temperature=0.8)

system_template = "You are a funny programmer and teacher that gives clean code based on instructions. " \
                  "You like to crack some witty jokes to make technical topics more interesting. " \
                  "Keep your response short."
human_template = "{content}"
# # Creates a chat prompt from the messages
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])
# Retrieves the LLM from the model config
# Creates a chain of chat models to process the message
chat_chain = chat_prompt | current_model | StrOutputParser()
# Invokes the chain with the message
response = chat_chain.invoke({"content":  "Hello. How are you?"})
print(response)


Hello! I'm doing great, thanks! Just hanging out in the cloud, making sure my code is cleaner than my desk. How can I help you today?


### Let's try connecting to local LLM via ollama

Step 1: Download ollama here - https://ollama.com/

Step 2: After download and installating ollama, let's setup llama3.1. This will download the llama3.1 models which may take some time.

ollama run llama3.1

Step 3: Once downloaded and initialized, you will receive a prompt like below

'>>> 

Step 4: Start typing any message like "How are you?". You should get a response from the model in your local computer.

In [15]:
from langchain_ollama.llms import OllamaLLM

# Let's try switching to Llama using Ollama
current_model = OllamaLLM(model='llama3.1', temperature=0.8)

system_template = "You are a funny programmer and teacher that gives clean code based on instructions. " \
                  "You like to crack some witty jokes to make technical topics more interesting. " \
                  "Keep your response short."
human_template = "{content}"
# # Creates a chat prompt from the messages
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])
# Retrieves the LLM from the model config
# Creates a chain of chat models to process the message
chat_chain = chat_prompt | current_model | StrOutputParser()
# Invokes the chain with the message
response = chat_chain.invoke({"content":  "Hello. How are you?"})
print(response)

I'm byte-ing into a great day, thanks for asking! Just got done debugging a few bugs – or should I say, bugs in the debugger?


### Let's ask the LLM to write some code

We can change the system prompt and user prompt to generate source code based on our desired behavior.

In [18]:

# current_model = ChatOpenAI(model="gpt-4o-mini",
#                          temperature=0.8)

current_model = OllamaLLM(model='llama3.1', temperature=0.8)

system_template = "You are a nextjs/typescript programmer that writes clean code based on standard coding convention. " \
                "Respond with the complete code in one file. " \
                "Apply change only on the specific issues reported. " \
                "Include the path of the new files in the response in the following format:\n " \
                "### Filename : <filename>\n" \
                "'''\n" \
                "<code>\n" \
                "'''"
human_template = "Create the files needed build the page described. {instruction}."

# Creates a chat prompt from the messages
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

# Retrieves the LLM from the model config
# Creates a chain of chat models to process the message
chat_chain = chat_prompt | current_model | StrOutputParser()
# Invokes the chain with the message
response = chat_chain.invoke({"instruction":  
                              "Create a login page with a form that has email and password fields. "})
print(response)



### components/LoginPage.tsx
```typescript
import React from 'react';

interface LoginFormProps {
  onLogin: (event: React.FormEvent<HTMLFormElement>) => void;
}

const LoginPage = ({ onLogin }: LoginFormProps) => {
  const handleLoginSubmit = (event: React.FormEvent<HTMLFormElement>) => {
    event.preventDefault();
    onLogin(event);
  };

  return (
    <div className="login-container">
      <form onSubmit={handleLoginSubmit}>
        <label>Email:</label>
        <input type="email" name="email" />
        <br />
        <label>Password:</label>
        <input type="password" name="password" />
        <button type="submit">Login</button>
      </form>
    </div>
  );
};

export default LoginPage;
```

### pages/login.tsx
```typescript
import React from 'react';
import LoginForm from '../components/LoginForm';

const LoginPage = () => {
  const handleLogin = (event: React.FormEvent<HTMLFormElement>) => {
    // Handle login logic here, for example:
    console.log('Handling login

In [20]:
# Let's save the results to files
files_created = []
state = 0 # 0: finding filename, 1: finding start content 2: finding end content
for message in response.split('\n'):
    if state == 0:
        if message.startswith('###'):
            filename = message[4:].replace('`', '').strip()
            print(f"Creating file: {filename}")
            content = ""
            state = 1
        else:
            print(f"{message}")
    elif state == 1:
        if message.startswith("```"):
            # we ignore this line
            state = 2
        else:
            print(f"{message}")
    elif state == 2:
        if message.startswith("```"):
            # end of content, lets write the content to file
            file_path = os.path.join('examples' ,filename)
            print(f"Writing content to file: {file_path}")
            files_created.append(file_path)
            # create the folder if it does not exist
            os.makedirs(os.path.dirname(file_path), exist_ok=True)
            with open(file_path, 'w') as f:
                f.write(content)
            state = 0
        else:
            content += message + "\n"

print(f"Files created: {files_created}")

Creating file: components/LoginPage.tsx
Writing content to file: examples/components/LoginPage.tsx

Creating file: pages/login.tsx
Writing content to file: examples/pages/login.tsx

Creating file: styles/globals.css
Writing content to file: examples/styles/globals.css
Note that I've kept the styling simple, you can add more complexity as needed. Also, remember to create a `components` and `pages` directory in your project root if they don't exist already.

Make sure to adjust the file paths according to your Next.js project structure.
Files created: ['examples/components/LoginPage.tsx', 'examples/pages/login.tsx', 'examples/styles/globals.css']


### We can also ask the LLM to update existing code...

In [22]:

current_model = OllamaLLM(model='llama3.1', temperature=0.8)

system_template = "You are a nextjs/typescript programmer that writes clean code based on standard coding convention. " \
    "Respond with the complete code in one file. " \
    "Apply change only on the specific issues reported. " \
    "Include the path of the new files in the response in the following format:\n " \
    "### Filename : <filename>\n" \
    "'''\n" \
    "<code>\n" \
    "'''"
human_template = "Update the code below with the given instruction.\n\n "\
                 "Code: \n ### Filename : {filename}\n"\
                 "'''{code}'''\n\n Instruction:{instruction}."

# Creates a chat prompt from the messages
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

with open('examples/components/LoginPage.tsx', 'r') as f:
    source_code = f.read()

# Retrieves the LLM from the model config
# Creates a chain of chat models to process the message
chat_chain = chat_prompt | current_model | StrOutputParser()
# Invokes the chain with the message
response = chat_chain.invoke({"instruction":
                              "Update email to username. ",
                              "filename":
                              "examples/components/LoginPage.tsx",
                              "code":
                              source_code})
print(response)

### Filename : examples/components/LoginPage.tsx
'''
import React from 'react';

interface LoginFormProps {
  onLogin: (event: React.FormEvent<HTMLFormElement>) => void;
}

const LoginPage = ({ onLogin }: LoginFormProps) => {
  const handleLoginSubmit = (event: React.FormEvent<HTMLFormElement>) => {
    event.preventDefault();
    onLogin(event);
  };

  return (
    <div className="login-container">
      <form onSubmit={handleLoginSubmit}>
        <label>Username:</label>
        <input type="username" name="username" />
        <br />
        <label>Password:</label>
        <input type="password" name="password" />
        <button type="submit">Login</button>
      </form>
    </div>
  );
};

export default LoginPage;
'''


In [23]:
# Let's save the results to files
files_created = []
state = 0 # 0: finding filename, 1: finding start content 2: finding end content
for message in response.split('\n'):
    if state == 0:
        if message.startswith('###'):
            filename = message[4:].replace('`', '').strip()
            print(f"Creating file: {filename}")
            content = ""
            state = 1
        else:
            print(f"{message}")
    elif state == 1:
        if message.startswith("```"):
            # we ignore this line
            state = 2
        else:
            print(f"{message}")
    elif state == 2:
        if message.startswith("```"):
            # end of content, lets write the content to file
            file_path = os.path.join('./' ,filename)
            print(f"Writing content to file: {file_path}")
            files_created.append(file_path)
            # create the folder if it does not exist
            os.makedirs(os.path.dirname(file_path), exist_ok=True)
            with open(file_path, 'w') as f:
                f.write(content)
            state = 0
        else:
            content += message + "\n"

print(f"Files created: {files_created}")

Creating file: Filename : examples/components/LoginPage.tsx
'''
import React from 'react';

interface LoginFormProps {
  onLogin: (event: React.FormEvent<HTMLFormElement>) => void;
}

const LoginPage = ({ onLogin }: LoginFormProps) => {
  const handleLoginSubmit = (event: React.FormEvent<HTMLFormElement>) => {
    event.preventDefault();
    onLogin(event);
  };

  return (
    <div className="login-container">
      <form onSubmit={handleLoginSubmit}>
        <label>Username:</label>
        <input type="username" name="username" />
        <br />
        <label>Password:</label>
        <input type="password" name="password" />
        <button type="submit">Login</button>
      </form>
    </div>
  );
};

export default LoginPage;
'''
Files created: []


### Let's invoke lint check to check for any errors and warnings

To keep the code clean and free from warnings and errors, we can perform lint check and let the LLM update the errors.

In this case, we have to use an external project that has already been setup with lint... if you have projects that uses lint, you can use that project instead.


In [24]:
import subprocess

try:
    working_dir = "/Users/mlmnl/Documents/GitHub/brad"
    lint_result = subprocess.run(
        ["npm", "run", "lint"],
        cwd=working_dir,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
    )
    lint_result = lint_result.stdout
    print(lint_result)
except subprocess.CalledProcessError as e:
    print("An error occurred while running lint check:")
    print("Exit Code:", e.returncode)
    print("Standard Error Output:", e.stderr)
    print("Standard Output:", e.stdout)


> brad@0.0.0 lint
> next lint


./src/app/(private)/admin/divisions/_hooks/use-column-divisions.tsx
115:5  Warning: React Hook useMemo has missing dependencies: 'openSheet', 'setAction', 'setSelectedRow', and 'setSelectedRowDetails'. Either include them or remove the dependency array.  react-hooks/exhaustive-deps

./src/app/(private)/admin/divisions/_hooks/use-form-divisions.ts
40:6  Warning: React Hook useEffect has a missing dependency: 'form'. Either include it or remove the dependency array.  react-hooks/exhaustive-deps

./src/app/(private)/admin/divisions/_hooks/use-option-divisions.ts
39:6  Warning: React Hook useEffect has missing dependencies: 'divisionOptions', 'parentDivisionOptions', 'setDivisions', and 'setParentDivisions'. Either include them or remove the dependency array.  react-hooks/exhaustive-deps

./src/app/(private)/admin/product/_hooks/use-column-products.tsx
92:5  Warning: React Hook useMemo has missing dependencies: 'openSheet', 'setAction', 'setSelectedRow', an

In [31]:
# Let's parse the result of lint check and let the LLM fix the issues
import re


lint_messages = {}
file_path = ""

# let's clean up the lint result first remove the strings in \x1b(.*?m)
lint_result = re.sub(r"\x1b\[.*?m", "", lint_result)
for line in lint_result.splitlines():
    if "error" in line.lower():
        lint_messages[file_path].append(line)
    elif "warning" in line.lower():
        lint_messages[file_path].append(line)
    else:
        file_path = line
        lint_messages[file_path] = []
# remove all empty lists
lint_messages = {k: v for k, v in lint_messages.items() if v}
print(lint_messages)

{'./src/app/(private)/admin/divisions/_hooks/use-column-divisions.tsx': ["115:5  Warning: React Hook useMemo has missing dependencies: 'openSheet', 'setAction', 'setSelectedRow', and 'setSelectedRowDetails'. Either include them or remove the dependency array.  react-hooks/exhaustive-deps"], './src/app/(private)/admin/divisions/_hooks/use-form-divisions.ts': ["40:6  Warning: React Hook useEffect has a missing dependency: 'form'. Either include it or remove the dependency array.  react-hooks/exhaustive-deps"], './src/app/(private)/admin/divisions/_hooks/use-option-divisions.ts': ["39:6  Warning: React Hook useEffect has missing dependencies: 'divisionOptions', 'parentDivisionOptions', 'setDivisions', and 'setParentDivisions'. Either include them or remove the dependency array.  react-hooks/exhaustive-deps"], './src/app/(private)/admin/product/_hooks/use-column-products.tsx': ["92:5  Warning: React Hook useMemo has missing dependencies: 'openSheet', 'setAction', 'setSelectedRow', and 'set

In [36]:
file_path = next(iter(lint_messages.keys()))
print(file_path)

./src/app/(private)/admin/divisions/_hooks/use-column-divisions.tsx


In [37]:
from pathlib import Path

def send_message(prompt: str, args: dict):
    merged_messages = []
    for message in prompt:
        # if message[1] is a list, join the list into a string
        if isinstance(message, dict):
            message = tuple(message.items())[0]
        merged_messages.append(message)

    # # Creates a chat prompt from the messages
    chat_prompt = ChatPromptTemplate.from_messages(merged_messages)
    # Retrieves the LLM from the model config
    # Creates a chain of chat models to process the message
    chat_chain = chat_prompt | current_model | StrOutputParser()
    # Invokes the chain with the message
    response = chat_chain.invoke(args)
    return response

def extract_messages(data: str):
    """
    Extracts messages from the provided data.
    """
    # Remove the code blocks from the data
    data = re.sub(r"```(?:[a-z]*)\n([\s\S]*?)```", "...", data)
    return data

def extract_code(data: str):
    """
    Extracts the code from the provided data.
    """
    code_pattern = re.compile(r"```(?:[a-z]*)\n([\s\S]*?)```", re.MULTILINE)

    code_match = code_pattern.search(data)

    if code_match:
        code = code_match.group(1).strip()
    else:
        code = ''

    return code

def write_code_to_file(file_path: str, code: str) -> str:
    """
    Writes the provided code to the specified file path, overwriting it if it exists.
    """
    # print(file_path)
    # print(code)
    if not file_path or not code:
        raise ValueError("File path or code not provided")

    path = Path(file_path)
    os.makedirs(path.parent, exist_ok=True)
    path.write_text(code+"\n")
    print(f"Code written to {path}")

    return str(file_path)

In [40]:
full_path = os.path.join(working_dir, file_path)
with open(full_path, 'r') as file:
    file_contents = file.read()
print(f"Fixing lint issues in {file_path} (attempt {attempt})...")
messages = lint_messages[file_path]
print(messages)


Fixing lint issues in ./src/app/(private)/admin/divisions/_hooks/use-column-divisions.tsx (attempt 1)...
["115:5  Warning: React Hook useMemo has missing dependencies: 'openSheet', 'setAction', 'setSelectedRow', and 'setSelectedRowDetails'. Either include them or remove the dependency array.  react-hooks/exhaustive-deps"]


In [41]:
current_model = OllamaLLM(model='llama3.1', temperature=0.8)

system_template = "You are a nextjs/typescript programmer and fixes lint issues in the code. " \
                "Respond with the complete code after fixing the issues. " \
                "Apply change only on the specific issues reported. " \
                "When introducing new lines of code, generate based on the following coding style:\n" \
                "1. Exclude extra semicolon in import statement.\n" \
                "### Filename : <filename>\n" \
                "'''\n" \
                "<code>\n" \
                "'''"

human_template ="Fix the following warnings or error: \n" \
                "{lint_issues}\n\n" \
                "Here is the code:" \
                "'''{code}'''"

# Creates a chat prompt from the messages
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

# Retrieves the LLM from the model config
# Creates a chain of chat models to process the message
chat_chain = chat_prompt | current_model | StrOutputParser()
# Invokes the chain with the message
response = chat_chain.invoke({"lint_issues":
                              " ".join(messages),
                              "code":
                              file_contents})
print(response)

Here is the code with the lint issues fixed:

```typescript
import useActionStore, { ACTION } from '@/store/use-action-store';
import { API_URL } from '@/enums/api';
import { Button } from '@/components/ui/button';
import { ColumnDef } from '@tanstack/react-table';
import { DataTableColumnHeader } from '@/components/data-table/data-table-column-header';
import { DataTableTextField } from '@/components/data-table/data-table-text-field';
import { DivisionWithRelations } from '@prisma/zod';
import { LuPencil } from 'react-icons/lu';
import { useMemo, useEffect } from 'react';
import useRowStore from '@/store/use-row-store';
import useSheetStore from '@/store/use-sheet-store';

const useColumnDivisions = () => {
  const openSheet = useSheetStore((s) => s.openSheet);
  const setAction = useActionStore((s) => s.setAction);
  const setSelectedRow = useRowStore((s) => s.setSelectedRow);
  const setSelectedRowDetails = useRowStore((s) => s.setSelectedRowDetails);

  useEffect(() => {
    return

### Save the results and overwrite the original file with the corrected code.

We will not perform the overwriting of file anymore, but I hope you get the idea behind using lint tools and LLM to automatically correct lint check errors